In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
!pip install sacrebleu

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install evaluate

In [ ]:
!pip install huggingface_hub

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os,gc,re,time
import matplotlib.pyplot as plt
import seaborn as sns
import spacy,random
import re,unicodedata
import time,datetime
from collections import Counter
import warnings,unicodedata
from datasets import load_dataset, load_metric
import evaluate
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
warnings.simplefilter("ignore")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Available device: {device}')

Available device: cuda


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

nltk.download('punkt')

# pre-processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("bentrevett/multi30k")
metric = load_metric("sacrebleu")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29000
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [ ]:
train_dataset = raw_datasets["train"]
validation_dataset = raw_datasets["validation"]
test_dataset = raw_datasets["test"]

In [ ]:
train_dataset

Dataset({
    features: ['en', 'de'],
    num_rows: 29000
})

In [ ]:
train_dataset.save_to_disk("/content/drive/MyDrive/Colab Notebooks/multi30k/train/train_ori")
test_dataset.save_to_disk("/content/drive/MyDrive/Colab Notebooks/multi30k/test/test_ori")
validation_dataset.save_to_disk("/content/drive/MyDrive/Colab Notebooks/multi30k/valid/validation_ori")

Saving the dataset (0/1 shards):   0%|          | 0/29000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1014 [00:00<?, ? examples/s]

In [ ]:
train = train_dataset[:]
train = pd.DataFrame(train)
test = test_dataset[:]
test = pd.DataFrame(test)
valid = validation_dataset[:]
valid = pd.DataFrame(valid)

In [ ]:
train.to_csv("/content/drive/MyDrive/Colab Notebooks/multi30k/train/train.tsv", sep='\t', index=False)
valid.to_csv("/content/drive/MyDrive/Colab Notebooks/multi30k/valid/valid.tsv", sep='\t', index=False)
test.to_csv("/content/drive/MyDrive/Colab Notebooks/multi30k/test/test.tsv", sep='\t', index=False)

In [ ]:
data_files = {"train": "/content/drive/MyDrive/Colab Notebooks/multi30k/train/train.tsv", "valid": "/content/drive/MyDrive/Colab Notebooks/multi30k/valid/valid.tsv", \
              "test": "/content/drive/MyDrive/Colab Notebooks/multi30k/test/test.tsv"}
dataset =  load_dataset("csv", data_files=data_files, delimiter="\t")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
model_ckpt = "t5-small"
max_token_length = 128

In [ ]:
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained(model_ckpt)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
dataset['train'][10]['de'], dataset['train'][10]['en']

('Eine Ballettklasse mit fünf Mädchen, die nacheinander springen.',
 'A ballet class of five girls jumping in sequence.')

In [ ]:
tokenized_sample_de = tokenizer(dataset['train'][10]['de'],
                                max_length=max_token_length,
                                padding=True, truncation=True)
tokenized_sample_de

{'input_ids': [2820, 23366, 17, 16424, 181, 9854, 21644, 6, 67, 559, 10517, 2141, 35, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenized_sample_en = tokenizer(dataset['train'][10]['en'],
                                max_length=max_token_length,
                                padding=True, truncation=True)
tokenized_sample_en

{'input_ids': [71, 22201, 853, 13, 874, 3567, 15539, 16, 5932, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
pd.DataFrame(
    [
        tokenized_sample_de['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_de['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
ids,2820,23366,17,16424,181,9854,21644,6,67,559,10517,2141,35,5,1
tokens,▁Eine,▁Ballet,t,klasse,▁mit,▁fünf,▁Mädchen,",",▁die,▁nach,einander,▁spring,en,.,</s>


In [ ]:
pd.DataFrame(
    [
        tokenized_sample_en['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_en['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,10
ids,71,22201,853,13,874,3567,15539,16,5932,5,1
tokens,▁A,▁ballet,▁class,▁of,▁five,▁girls,▁jumping,▁in,▁sequence,.,</s>


In [ ]:
# tokenizer를 이용해 문자로 된 문장을 숫자로 바꿔 특성화

def convert_examples_to_features(examples):
    model_inputs = tokenizer(examples['de'],
                             text_target=examples['en'],
                             max_length=max_token_length, truncation=True)

    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(convert_examples_to_features,
                                 batched=True,
                                 remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 29000
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [ ]:
tokenized_datasets['train'][10]

In [ ]:
print( '원 데이터    :', dataset['train'][10]['de'] )
print( '처리 후 데이터:', tokenized_datasets['train'][10]['input_ids'] )
print( '토큰화       :', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['input_ids']) )

print('\n')
print( '원 데이터    :', dataset['train'][10]['en'] )
print( '처리 후 데이터:', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['labels']) )
print( '토큰화       :', tokenized_datasets['train'][10]['labels'] )

원 데이터    : Eine Ballettklasse mit fünf Mädchen, die nacheinander springen.
처리 후 데이터: [2820, 23366, 17, 16424, 181, 9854, 21644, 6, 67, 559, 10517, 2141, 35, 5, 1]
토큰화       : ['▁Eine', '▁Ballet', 't', 'klasse', '▁mit', '▁fünf', '▁Mädchen', ',', '▁die', '▁nach', 'einander', '▁spring', 'en', '.', '</s>']


원 데이터    : A ballet class of five girls jumping in sequence.
처리 후 데이터: ['▁A', '▁ballet', '▁class', '▁of', '▁five', '▁girls', '▁jumping', '▁in', '▁sequence', '.', '</s>']
토큰화       : [71, 22201, 853, 13, 874, 3567, 15539, 16, 5932, 5, 1]


# Model

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model_ckpt).to(device)

In [ ]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
encoder_inputs = tokenizer(
    ["Wäre es angemessen, wenn Sie, Frau Präsidentin, der Präsidentin von Sri Lanka in einem Schreiben das Bedauern des Parlaments zum gewaltsamen Tod von Herrn Ponnambalam und anderen Bürgern von Sri Lanka übermitteln \
      und sie auffordern würden, alles in ihrem Kräften stehende zu tun, um nach einer friedlichen Lösung dieser sehr schwierigen Situation zu suchen?"],
    return_tensors="pt"
)['input_ids'].to(device)

decoder_targets = tokenizer(
    ["Would it be appropriate for you, Madam President, to write a letter to the Sri Lankan President expressing Parliament's regret at his and the other violent deaths in Sri Lanka and urging her to do everything she possibly \
      can to seek a peaceful reconciliation to a very difficult situation?"],
    return_tensors="pt"
)['input_ids'].to(device)

In [ ]:
print( encoder_inputs )
print( decoder_targets )

tensor([[  549,  3185,    15,     3,    15,     7,     3,  3280, 14202,     6,
          1301,   292,     6,  7672,     3, 20962,    77,     6,    74,     3,
         20962,    77,   193,  8642, 13877,    16,   665, 27735,   211,   493,
         15069,    29,    93, 13636,     7,   674,   873,  5380,    17,     7,
             9,   904,   304,    26,   193,  8816,    29, 20093, 13363, 10810,
            51,    64,  2778, 11726,    29,   193,  8642, 13877,   510,  5401,
            29,    64,   680,   219, 21390, 15332,     6,  3532,    16,  1159,
            51, 31436,    29,     3, 21256,   170,  5240,     6,   561,   559,
           645,     3,  7704,  2978, 13697,     3,  1878,  1319, 19002,    35,
         15494,   170, 13748,    58,     1]], device='cuda:0')
tensor([[ 5328,    34,    36,  2016,    21,    25,     6,  5428,   265,  1661,
             6,    12,  1431,     3,     9,  2068,    12,     8,  8642, 13877,
            29,  1661,     3, 21008, 12876,    31,     7,  9867,    

In [ ]:
# 디코더 출력을 오른쪽으로 쉬프트시키는 것 - 디코더 입력값
decoder_inputs = model._shift_right(decoder_targets)

In [ ]:
pd.DataFrame(
    [
        tokenizer.convert_ids_to_tokens(decoder_targets[0]),
        tokenizer.convert_ids_to_tokens(decoder_inputs[0])
    ],
    index=('decoder target', 'decoder input')
)
# decoder input의 경우 <pad>가 추가

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
decoder target,▁Would,▁it,▁be,▁appropriate,▁for,▁you,",",▁Mad,am,▁President,...,▁peaceful,▁reconciliation,▁to,▁,a,▁very,▁difficult,▁situation,?,</s>
decoder input,<pad>,▁Would,▁it,▁be,▁appropriate,▁for,▁you,",",▁Mad,am,...,a,▁peaceful,▁reconciliation,▁to,▁,a,▁very,▁difficult,▁situation,?


In [ ]:
outputs = model(input_ids=encoder_inputs,
                decoder_input_ids=decoder_inputs,
                labels=decoder_targets)

In [ ]:
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

In [ ]:
outputs.loss
# grad_fn은 텐서가 어떤 연산을 하였는지에 대한 정보를 담고 있으며, 역전파 과정에서 사용

tensor(3.0807, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
outputs['encoder_last_hidden_state'].shape
# sample, step, model_size

torch.Size([1, 95, 512])

In [ ]:
outputs['logits'].shape

torch.Size([1, 62, 32128])

In [ ]:
# 예제 배치를 생성하는 코드로, 데이터 세트의 최대 길이로 padding -> 다른 시퀀스 lenght를 가진 input을 합쳐주는 것

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator(
    [tokenized_datasets["train"][i] for i in range(1, 3)]
)

In [ ]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
batch
# labels에 있는 paddding token을 -100으로 설정해 손실 함수가 무시하도록 만듦

{'input_ids': tensor([[11093,    49,    15, 16282,   181, 10956, 17757,    35,  1953,    23,
            35,    35,   236,   389, 13835,     7,  5672,  3734,     5,     1],
        [  890,  5683,     7, 21644,     3,   157, 16024,    17,    16,   236,
          3012,  2989,   403,  8589,     5,     1,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]]), 'labels': tensor([[    3,  8656,  1076,    16,   614,     3,   547,     7,    33,  2699,
             3,     9,  6079,  3197,    15,    63,   358,     5,     1],
        [   71,   385,  3202, 11908,   139,     3,     9,  5726,   577,  1840,
             5,     1,  -100,  -100,  -100,  -100,  -100,  -100,  -100]]), 'decoder_input_ids': tensor([[    0,     3,  8656,  1076,    16,   614,     3,   547,     7,    33,
          2699,     3,     9,  6079,  3197,    15,    63,   358,     5],
        [    0,  

# Metric

In [ ]:
metric = evaluate.load("sacrebleu")

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # 모델이 예측 로짓(logits)외의 다른 것을 return하는 경우
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # 레이블 (정답)에서는 모든 -100을 제외
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    return result


# Training

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="chkpt",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=10,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_strategy="no",
    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=2,
    report_to="none"
)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
0,No log,1.515619,24.068517
2,No log,1.321944,30.287495
4,No log,1.250014,31.993613
6,No log,1.215983,33.074015
8,No log,1.199753,33.362655
9,No log,1.198127,33.307679


TrainOutput(global_step=12080, training_loss=1.6043720573778975, metrics={'train_runtime': 1581.9689, 'train_samples_per_second': 183.316, 'train_steps_per_second': 7.636, 'total_flos': 2587594615554048.0, 'train_loss': 1.6043720573778975, 'epoch': 10.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/multi30k/results")

In [ ]:
model_dir = "/content/drive/MyDrive/Colab Notebooks/multi30k/results"
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)

model.cpu();

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Translate

In [ ]:
# 1. 텍스트를 토큰화하고 input_ids를 Pytorch tensor로 변환
# 2. generate()로 번역을 생성
# 3. 생성된 토큰 ID를 다시 텍스트로 디코

In [ ]:
input_text = [
    'Wäre es angemessen, wenn Sie, Frau Präsidentin, der Präsidentin von Sri Lanka in einem Schreiben das Bedauern des Parlaments zum gewaltsamen Tod \
    von Herrn Ponnambalam und anderen Bürgern von Sri Lanka übermitteln und sie auffordern würden, alles in ihrem Kräften stehende zu tun, um nach einer \
     friedlichen Lösung dieser sehr schwierigen Situation zu suchen?'
]

In [ ]:
inputs = tokenizer(input_text, return_tensors="pt",
                   padding=True, max_length=max_token_length)

In [ ]:
inputs

{'input_ids': tensor([[  549,  3185,    15,     3,    15,     7,     3,  3280, 14202,     6,
          1301,   292,     6,  7672,     3, 20962,    77,     6,    74,     3,
         20962,    77,   193,  8642, 13877,    16,   665, 27735,   211,   493,
         15069,    29,    93, 13636,     7,   674,   873,  5380,    17,     7,
             9,   904,   304,    26,   193,  8816,    29, 20093, 13363, 10810,
            51,    64,  2778, 11726,    29,   193,  8642, 13877,   510,  5401,
            29,    64,   680,   219, 21390, 15332,     6,  3532,    16,  1159,
            51, 31436,    29,     3, 21256,   170,  5240,     6,   561,   559,
           645,     3,  7704,  2978, 13697,     3,  1878,  1319, 19002,    35,
         15494,   170, 13748,    58,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
English = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

English.shape

torch.Size([1, 58])

In [ ]:
# 생성된 결과를 디코딩

[tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(Englishs)) for Englishs in English]

['<pad> You, lady president, the president of Sri Lanka in a letter to the waltsamen tod of Mr Ponnambalam and other citizens of Sri Lanka, are asking for everything in their efforts to find a peaceful solution to this very difficult situation?</s>']

In [ ]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=32, collate_fn=data_collator
)

In [ ]:
test_dataloader_iter = iter(test_dataloader)
test_batch = next(test_dataloader_iter)

In [ ]:
test_batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
test_input = { key: test_batch[key] for key in ('input_ids', 'attention_mask') }

In [ ]:
English_1 = model.generate(
    **test_input,
    max_length=max_token_length,
    num_beams=5,
)

In [ ]:
# -100으로 인코딩된 부분을 padding token으로 교체

labels =  np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)

In [ ]:
Gen_sents = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens=True)[10:20]

In [ ]:
references = tokenizer.batch_decode(labels, skip_special_tokens=True)[10:20]

In [ ]:
preds = tokenizer.batch_decode( English_1, skip_special_tokens=True )[10:20]

In [ ]:
for s in zip(Gen_sents, references, preds):
    print('German   :', s[0])  # 입력 문장
    print('Reference :', s[1])  # 정답
    print('Translated:', s[2])  # 모델의 번역 문장
    print('\n')

German   : Eine Mutter und ihr kleiner Sohn genießen einen schönen Tag im Freien.
Reference : A mother and her young song enjoying a beautiful day outside.
Translated: A mother and her little son enjoy a nice day outside.


German   : Männer, die Volleyball spielen, wobei ein Mann den Ball nicht trifft, während seine Hände immer noch in der Luft sind.
Reference : Men playing volleyball, with one player missing the ball but hands still in the air.
Translated: Men playing volleyball while a man is playing the ball while his hands are still in the air.


German   : Eine Frau, die in einer Küche eine Schale mit Essen hält.
Reference : A woman holding a bowl of food in a kitchen.
Translated: A woman holding a bowl of food in a kitchen.


German   : Ein sitzender Mann, der an einem Tisch in seinem Haus mit einem Werkzeug arbeitet.
Reference : Man sitting using tool at a table in his home.
Translated: A man sitting at a table with a tool in his house.


German   : Drei Leute sitzen in einer H

In [ ]:
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=32, collate_fn=data_collator
)

for test_batch in test_dataloader:
    test_input = {key: test_batch[key] for key in ('input_ids', 'attention_mask')}
    English_1 = model.generate(
        **test_input,
        max_length=max_token_length,
        num_beams=5,
    )
    labels = np.where(test_batch['labels'] != -100, test_batch['labels'], tokenizer.pad_token_id)

    Gen_sents = tokenizer.batch_decode(test_batch['input_ids'], skip_special_tokens=True)[10:20]
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)[10:20]
    preds = tokenizer.batch_decode(English_1, skip_special_tokens=True)[10:20]

    for s in zip(Gen_sents, references, preds):
       print('German   :', s[0])  # 입력 문장
       print('Reference :', s[1])  # 정답
       print('Translated:', s[2])  # 모델의 번역 문장
       print('\n')


In [ ]:
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=32, collate_fn=data_collator
)

bleu_scores = []

for test_batch in test_dataloader:
    test_input = {key: test_batch[key] for key in ('input_ids', 'attention_mask')}
    English_1 = model.generate(
        **test_input,
        max_length=max_token_length,
        num_beams=5,
    )
    labels = np.where(test_batch['labels'] != -100, test_batch['labels'], tokenizer.pad_token_id)

    Gen_sents = tokenizer.batch_decode(test_batch['input_ids'], skip_special_tokens=True)
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)
    preds = tokenizer.batch_decode(English_1, skip_special_tokens=True)

    for s in zip(Gen_sents, references, preds):
        print('German   :', s[0])  # 입력 문장
        print('Reference :', s[1])  # 정답
        print('Translated:', s[2])  # 모델의 번역 문장

        # Calculate BLEU-4 score
        reference_sentence = s[1].split()
        translate_sentence = s[2].split()
        bleu_score = sentence_bleu([reference_sentence], translate_sentence, weights=(0.25, 0.25, 0.25, 0.25))
        bleu_scores.append(bleu_score)

        print('BLEU-4 Score:', bleu_score)  # BLEU-4 점수 출력
        print('\n')

average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print('Average BLEU-4 Score:', average_bleu_score)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

German   : Ein Hund, der einen Mantel trägt, läuft im Schnee.
Reference : A dog wearing a cover runs in the snow.
Translated: A dog wearing a coat runs in the snow.
BLEU-4 Score: 0.5969491792019646


German   : Eine Familie spielt am Strand mit ihrem Hund.
Reference : A family is playing on the beach with their dog.
Translated: A family plays with their dog on the beach.
BLEU-4 Score: 9.438484684416467e-155


German   : Eine Frau und ein Kind umarmen sich während das Kind kostet, was sie gerade gemeinsam zubereiten.
Reference : A woman and a child embrace while the child tastes what they have been making.
Translated: A woman and a child are talking while the child costs.
BLEU-4 Score: 0.3261942074256506


German   : Zwei Männer und eine Dame stehen im Freien.
Reference : Two men and a lady are standing outside.
Translated: Two men and a lady standing outside.
BLEU-4 Score: 0.6129752413741056


German   : Eine Gruppe von Männern verwendet Baumstämme,